In [ ]:
# Directory of schools
schools_csv_map = [
    {"school_1": "PILTON COMMUNITY COLLEGE",
     "school_2": "GREAT TORRINGTON SCHOOL",
     "school_3": "BRAUNTON SCHOOL AND C.C.",
     "school_4": "HOLSWORTHY COMMUNITY COLLEGE",
     "school_5": "THE ILFRACOMBE ACADEMY"}
]

In [ ]:
# Read in data



In [ ]:
# Make any changes required (e.g. handling missing data, dropping columns), perform any validation steps

In [ ]:
# Generate graphs

# Section A graphs: summarise RAG ratings for all pupils as a table


In [ ]:
# Section B graphs: for each topic, generate bar chart and comparison to other schools

In [ ]:
# Section C graphs: who took part

In [ ]:
# Save outputs 

save [tables, graphs...]